<a href="https://colab.research.google.com/github/aidantze/pesta-la-vista/blob/yolo_v8_dis/yolov8_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q ultralytics kagglehub pyyaml

import kagglehub
import pathlib
import yaml
import shutil
from ultralytics import YOLO
import torch

# Check for T4 availability on Colab (DON'T CHANGE)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))
else:
    print("No GPU detected. Go to Runtime -> Change runtime type ->  GPU")

# Kaggle Download via CLI API (see their website - DON'T CHANGE)
path = kagglehub.dataset_download("rupankarmajumdar/crop-pests-dataset")

# Saved the images to a local path to increase efficiency
local_path = pathlib.Path("/content/datasets/crop-pests")
local_path.parent.mkdir(parents=True, exist_ok=True)
shutil.copytree(path, local_path, dirs_exist_ok=True)

# YAML CONGIF (DON'T CHANGE)
data_yaml_path = local_path / "data.yaml"
data_cfg = {
    "path": str(local_path),
    "train": "train/images",
    "val":   "valid/images",
    "test":  "test/images",
    "names": [
        "ant", "bee", "beetle", "caterpillar", "earthworm", "earwig",
        "grasshopper", "moth", "slug", "snail", "wasp", "weevil"
    ]
}

with open(data_yaml_path, "w") as f:
    yaml.safe_dump(data_cfg, f)

# Training the Model (as per YOLOv8n website)
model = YOLO("yolov8n.pt")

# See changes made to increase for Colab
train_res = model.train(
    data=str(data_yaml_path),
    epochs=30,
    imgsz=512,
    batch=-1,            # auto batch size based on GPU memory
    device=0,            # use GPU (CUDA:0)
    workers=4,           # increase data loading threads
    cache=True,          # cache dataset in RAM/Disk for speed
    amp=True,            # mixed precision
    freeze=10,           # aka for transfer learning (recommended by AI)
    mosaic=0.5,          # light augmentations
    mixup=0.0,
    copy_paste=0.0,
    project="pests_fast",
    name="yolov8n_colab",
    plots=False
)

# Validate the Model (as per YOLOv8n website)
val_results = model.val(
    data=str(data_yaml_path),
    split="test",
    batch=16,
    device=0,
    workers=2
)
print(val_results)

# Test the Model (as per YOLOv8n website)
sample_dir = local_path / "test" / "images"
model.predict(
    source=str(sample_dir),
    batch=16,
    imgsz=512,
    device=0,
    save=True,
    project="runs/detect",
    name="pest_predictions"
)